**Raw data**


In [98]:
import pandas as pd

data = [
    [101, '"Widget A"', "$10.50", "USA"],
    [102, "Widget B", "$5.00", "Canada"],
    [103, "Widget A", "10.50", "USA"]
]

df = pd.DataFrame(data)
df.to_csv("sales.csv", header=False, index=False)


**Read CSV**


In [99]:
import pandas as pd

USD_TO_INR = 83

df = pd.read_csv("sales.csv", header=None,
                 names=["id", "product", "price", "country"])

In [100]:
df

,id,product,price,country
0,101,"""Widget A""",$10.50,USA
1,102,Widget B,$5.00,Canada
2,103,Widget A,10.50,USA


**Clean columns**




In [101]:
# Remove irrelevant " in product column
df["product"] = df["product"].str.replace('"', '', regex=False).str.strip()


In [102]:
df

,id,product,price,country
0,101,Widget A,$10.50,USA
1,102,Widget B,$5.00,Canada
2,103,Widget A,10.50,USA


In [103]:
#remove $ in price column
df["price"] = (df["price"].str.replace("$", "", regex=False).astype(float))

In [104]:
df

,id,product,price,country
0,101,Widget A,10.5,USA
1,102,Widget B,5.0,Canada
2,103,Widget A,10.5,USA


**Remove  Deduplicate (same product + price)**

In [105]:
print(f'Total number of duplicate rows: {df.duplicated(subset=["product", "price"]).sum()}')
df = df.drop_duplicates(subset=["product", "price"])
print(f'Total number of rows after removing duplicates: {len(df)}')

Total number of duplicate rows: 1
Total number of rows after removing duplicates: 2


In [106]:
df

,id,product,price,country
0,101,Widget A,10.5,USA
1,102,Widget B,5.0,Canada


**Convert currency**


In [107]:
# convert in INR and Round the price
df["price_inr"] = (df["price"] * USD_TO_INR).round(2)

# Rename for clarity
df = df.rename(columns={"price": "price_usd"})

/tmp/ipython-input-1181506323.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["price_inr"] = (df["price"] * USD_TO_INR).round(2)


In [108]:
# Ensure the data before convert into JSON
df

,id,product,price_usd,country,price_inr
0,101,Widget A,10.5,USA,871.5
1,102,Widget B,5.0,Canada,415.0


**Save in JSON**


In [109]:

# Save cleaned data to a new JSON file
df.to_json("clean_sales.json", orient="records", indent=4)
print("clean_sales.json created successfully!")


clean_sales.json created successfully!


**Check Code in One go for creating a python file**




In [110]:
#import libraries
import pandas as pd

USD_TO_INR = 83 # Conversion rate from USD to INR (Given in the question)

def main(input_file="sales.csv", output_file="clean_sales.json"):
    """Clean messy sales CSV and output JSON report."""

    print("Reading sales.csv...")

    df = pd.read_csv(input_file, header=None,
                    names=["id", "product", "price", "country"],
                    encoding="utf-8",
                    skipinitialspace=True)

    print(f'Total number of rows in original data: {len(df)}')


    # Remove irrelevant " in product column
    df["product"] = df["product"].str.replace('"', '', regex=False).str.strip()

    #remove $ in price column
    df["price"] = (df["price"].str.replace("$", "", regex=False)
                   .str.strip().astype(float))

    # Remove rows with missing values
    print('Number of null values per column:')
    print(df.isnull().sum())
    df = df.dropna()
    print(f'Total number of rows after removing null values: {len(df)}')

    #check for duplicates and remove them
    print(f'Total number of duplicate rows: {df.duplicated(subset=["product", "price"]).sum()}')
    df = df.drop_duplicates(subset=["product", "price"])
    print(f'Total number of rows after removing duplicates: {len(df)}')

    # Convert price from USD to INR
    df["price_inr"] = (df["price"] * USD_TO_INR).round(2)


    # Rename for clarity
    df = df.rename(columns={"price": "price_usd"})
    # Reorder columns
    df = df[["id", "product", "price_usd", "price_inr", "country"]]


    # Save cleaned data to a new JSON file
    df.to_json(output_file, orient="records", indent=4)
    print(f"{output_file} created successfully!")

if __name__ == "__main__":
    main("sales.csv", "clean_sales.json")

Reading sales.csv...
Total number of rows in original data: 3
Number of null values per column:
id         0
product    0
price      0
country    0
dtype: int64
Total number of rows after removing null values: 3
Total number of duplicate rows: 1
Total number of rows after removing duplicates: 2
clean_sales.json created successfully!
